In [1]:
from PIL import Image
import os
import time
import sys
import random
import shutil
import re


In [3]:
%load_ext autoreload
%autoreload 2

#jupyter 中开启该选项，否则不执行
%matplotlib inline

# 数据集随机提取

pick_hazy_and_gt_img()函数输出到目标地址后的文件命名规则为：雾图为原来无雾图片的名字， 无雾图片为雾图名 ＋ '_gt'

In [ ]:
def pick_hazy_and_gt_img(src_hazy_dir_path, src_gt_dir_path, dst_hazy_dir_path, dst_gt_dir_path, number, suffix, random_tag=1):

    
    gt_img_list = os.listdir(src_gt_dir_path)
    
    # shuffle the list of the dir
    if random_tag == 1:
    
        random.shuffle(gt_img_list)
    
    # get number imgs
    gt_img_list = gt_img_list[0:number]
    
    num_gt_copy_before = len(os.listdir(dst_gt_dir_path))
    
    num_hazy_copy_before = len(os.listdir(dst_hazy_dir_path))    
    
    
    for gt_img in gt_img_list:
        (gt_img_name, gt_img_form) = gt_img.split('.',-1)
        
        hazy_img = gt_img_name + suffix + '.' + gt_img_form

        src_hazy_img_path = os.path.join(src_hazy_dir_path, hazy_img)
        
        src_gt_img_path = os.path.join(src_gt_dir_path, gt_img)

        dst_hazy_img = gt_img
        
        dst_gt_img = gt_img_name + '_gt.' + gt_img_form
        
        dst_hazy_img_path = os.path.join(dst_hazy_dir_path, dst_hazy_img)
        
        dst_gt_img_path = os.path.join(dst_gt_dir_path, dst_gt_img)

        # copy the src dataset to evaluate folder
        # shutil.copyfile(src, dst)
        shutil.copyfile(src_hazy_img_path, dst_hazy_img_path)
        
        shutil.copyfile(src_gt_img_path, dst_gt_img_path)

    num_gt_copy_now = len(os.listdir(dst_gt_dir_path))
    
    num_hazy_copy_now = len(os.listdir(dst_hazy_dir_path))
    
    num_gt_copy = num_gt_copy_now - num_gt_copy_before
    
    num_hazy_copy_now = num_hazy_copy_now - num_hazy_copy_before
    
    print("Successfully copy the source dataset to evaluate folder!")
    
    print("%d expected, %d groundtruth image done, %d hazy image done" %(number, num_gt_copy, num_hazy_copy_now))

In [ ]:
def pick_img(src_hazy_dir_path, src_gt_dir_path, dst_hazy_dir_path, dst_gt_dir_path, number,random_tag=1):
    
    
    hazy_img_list = os.listdir(src_hazy_dir_path)
   
    # shuffle the list of the dir
    if random_tag == 1:
        
        random.shuffle(hazy_img_list)
        
    # get number imgs
    hazy_img_list = hazy_img_list[0:number]
    
    num_gt_copy_before = len(os.listdir(dst_gt_dir_path))
    
    num_hazy_copy_before = len(os.listdir(dst_hazy_dir_path))    
    
    
    for hazy_img in hazy_img_list:

        gt_img = hazy_img[0:4] + '.png'
        
        src_hazy_img_path = os.path.join(src_hazy_dir_path, hazy_img)
        
        src_gt_img_path = os.path.join(src_gt_dir_path, gt_img)

        dst_hazy_img = gt_img
        
        dst_gt_img = hazy_img[0:4] + '_gt' + '.png'
        
        dst_hazy_img_path = os.path.join(dst_hazy_dir_path, dst_hazy_img)
        
        dst_gt_img_path = os.path.join(dst_gt_dir_path, dst_gt_img)

        # copy the src dataset to evaluate folder
        # shutil.copyfile(src, dst)
        shutil.copyfile(src_hazy_img_path, dst_hazy_img_path)
        
        shutil.copyfile(src_gt_img_path, dst_gt_img_path)

    
    num_gt_copy_now = len(os.listdir(dst_gt_dir_path))
    
    num_hazy_copy_now = len(os.listdir(dst_hazy_dir_path))
    
    num_gt_copy = num_gt_copy_now - num_gt_copy_before
    
    num_hazy_copy_now = num_hazy_copy_now - num_hazy_copy_before
    
    print("Successfully copy the source dataset to evaluate folder!")
    
    print("%d expected, %d groundtruth image done, %d hazy image done" %(number, num_gt_copy, num_hazy_copy_now))

### 随机从NYU2 数据集中抽取 number个 无雾图片及其对应的雾图

In [ ]:
if __name__ == '__main__':
    src_hazy_dir_path = r'.\data\RESIDE\SOTS\SOTS\outdoor\hazy'
    src_gt_dir_path = r'.\data\RESIDE\SOTS\SOTS\outdoor\gt'
    dst_hazy_dir_path = './Graduation_project_defog/evaluate/hazy'
    dst_gt_dir_path = './Graduation_project_defog/evaluate/groundtruth'
    number = 1
    suffix = '_7_2'
    pick_hazy_and_gt_img(src_hazy_dir_path, src_gt_dir_path, dst_hazy_dir_path, dst_gt_dir_path, number, suffix=0, random_tag=1)
    # pick_img(src_hazy_dir_path, src_gt_dir_path, dst_hazy_dir_path, dst_gt_dir_path, number,random_tag=1)

In [ ]:
def test_pair(hazy_dir, gt_dir):
    hazy_img_list = os.listdir(hazy_dir)
    gt_img_list = os.listdir(gt_dir)
    for hazy_img in hazy_img_list:
        gt_img = hazy_img[0:4] + '_gt.png'
        if gt_img not in gt_img_list:
            print('error:%s' %hazy_img)
    
    print('No problems!')

In [ ]:
if __name__ == '__main__':
    hazy_dir = r'.\evaluate\hazy'
    gt_dir = r'.\evaluate\groundtruth'
    test_pair(hazy_dir, gt_dir)

# 图片裁剪

In [ ]:
import cv2

In [ ]:
# change the size of image
# after this operation, w, h meet the condition of w%16==0. h%16==0
def preprocess_img(img_dir_path):
    img_list = os.listdir(img_dir_path)
    num_img = len(img_list)
    cutting_count = 0
    for img in img_list:
        img_path = os.path.join(img_dir_path,img)
        src = cv2.imread(img_path)    #src.shape == [h,w,c]
        (h,w) = src.shape[0:2]
        if w%PATCH_SIZE != 0 or h%PATCH_SIZE != 0:
            if w%PATCH_SIZE != 0:
                w -= (w%PATCH_SIZE)
            if h%PATCH_SIZE != 0:
                h -= (h%PATCH_SIZE)
            src = src[0:h, 0:w]  # 裁剪坐标为[y0:y1, x0:x1]
            cv2.imwrite(img_path,src)
            cutting_count += 1
    print("Preprocessing Done! Total %d images, where %d cut" %(num_img, cutting_count))


In [ ]:
if __name__ == '__main__':
    PATCH_SIZE = 16
    needprocess = ['hazy']
    for name in needprocess:
        dir_path = r'.\evaluate\\' + name
        preprocess_img(dir_path)
        print("process done:%s" %name)

# 目录创建

In [4]:
def dir_create(root_path, folder_list):
    
    if not os.path.exists(root_path):
        
        os.mkdir(root_path)
        
        print("New create root folder...")
    
    for folder_name in folder_list:
        
        folder_path = os.path.join(root_path,folder_name)
        
        if not os.path.exists(folder_path):
            
            os.mkdir(folder_path)
            
            print('New create folder: %s' %folder_name)
            
        else:
            
            print('Already existing folder: %s' %folder_name)
            
    print('Folder dir creating done!')

In [7]:
if __name__ == '__main__':
    
    root_path = r'.\evaluate'
    
    #evaluate_dataset_name = 'NYU2_200_random'
    folder_list = ['hazy', 'groundtruth', 'proposed','result', 
                   'ATM','BCCR', 'CAP', 'DCP', 'DehazeNet', 'FVR', 'MSCNN','NLD']
    
    #dir_create(root_path, evaluate_dataset_name, folder_list)
    dir_create(root_path, folder_list)

New create folder: hazy
New create folder: groundtruth
New create folder: proposed
New create folder: result
New create folder: ATM
New create folder: BCCR
New create folder: CAP
New create folder: DCP
New create folder: DehazeNet
New create folder: FVR
New create folder: MSCNN
New create folder: NLD
Folder dir creating done!


# MSD 图像加雾

根据MSD深度信息合成对应雾图。

根据alpha beta 值生成雾图


In [ ]:
import glob

import cv2

import numpy as np

In [ ]:
def haze_generator(msd_dir_path, output_dir_path, generator_num_per_img=5):

    if not os.path.exists(output_dir_path):
        
        os.mkdir(output_dir_path)
        
        print("New create output dir folder:%s" % output_dir_path)
 
    img_name_alphabeta = {}
    
    folders_tofind = msd_dir_path + '/*'
    
    folders = glob.glob(folders_tofind)
    
    for folder in folders:
        
        limg = cv2.imread(folder+'/view1.png', 1)/255.0
        
        ldis = cv2.imread(folder+'/disp1.png', 0)/3
        
        rimg = cv2.imread(folder+'/view5.png', 1)/255.0
        
        rdis = cv2.imread(folder+'/disp5.png', 0)/3
        
        for i in range(1, ldis.shape[0]):
            
            for j in range(ldis.shape[1]):
                
                if (ldis[i,j] == 0):
                    
                    ldis[i, j] = ldis[i-1, j]
                    
                if (rdis[i,j] == 0):
                    
                    rdis[i, j] = rdis[i-1, j]
                    
        for i in range(ldis.shape[0]):
            
            for j in range(1, ldis.shape[1]):
                
                if (ldis[i,j] == 0):
                    
                    ldis[i, j] = ldis[i, j-1]
                    
                if (rdis[i,j] == 0):
                    
                    rdis[i, j] = rdis[i, j-1]
                    
        ldis = np.clip(ldis, 1, ldis.shape[1])
        
        rdis = np.clip(rdis, 1, rdis.shape[1])
        
        ldepth = 3740*0.016/ldis
        
        rdepth = 3740*0.016/rdis
    
        for c in range(generator_num_per_img):
            
            alpha = random.uniform(0.75, 1.0)
            
            beta = random.uniform(0, 1.5)
            
            lt = np.exp(-beta*cv2.merge((ldepth, ldepth, ldepth)))
            
            rt = np.exp(-beta*cv2.merge((rdepth, rdepth, rdepth)))
            
            lI = 255*(np.multiply(limg,lt)+alpha*(1-lt))
            
            rI = 255*(np.multiply(rimg,rt)+alpha*(1-rt))
            
            lI = lI.astype(np.uint8)
            
            rI = rI.astype(np.uint8)
            
            name = folder.split('\\')[-1]
            
            alphabeta_info = 'alpha:  ' + str(alpha) + '   beta:  ' + str(beta)      

            img_name1 = name+'_'+str(c)+'_1.png'
            
            img_name5 = name+'_'+str(c)+'_5.png'
            
            img_name_alphabeta[img_name1] = alphabeta_info
            
            img_name_alphabeta[img_name5] = alphabeta_info
            
            cv2.imwrite(output_dir_path + '/' + img_name1, lI)
            
            cv2.imwrite(output_dir_path + '/' + img_name5, rI)

    info_path = os.path.join(output_dir_path,'alphabeta_info.txt')
    
    with open(info_path, 'w') as f:
        
        for img_name in img_name_alphabeta:
            
            f.write('%s           :' % str(img_name))
            
            f.write(str(img_name_alphabeta[img_name])+ '\n')
    
    print("That's all...")

In [ ]:
if __name__ == '__main__':
    
    msd_dir_path = r'.\data\middlebury_2006'
    
    output_dir_path = r'.\data\middlebury_2006_hazy'
    
    generator_num_per_img=1  # 控制生成图数量
    
    haze_generator(msd_dir_path, output_dir_path, generator_num_per_img)